<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/roboflow_bccd_hf_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q roboflow datasets datasets[vision]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is

In [2]:
from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-100").project("bccd-ouzjz")
version = project.version(2)
dataset = version.download(model_format="coco", location="/content/coco/", overwrite=True)
dataset = version.download(model_format="paligemma", location="/content/paligemma/", overwrite=True)
dataset = version.download(model_format="florence2-od", location="/content/florence2_od/", overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/coco/ in coco:: 100%|██████████| 372/372 [00:00<00:00, 6775.20it/s]


Extracting Dataset Version Zip to /content/paligemma/ in paligemma:: 100%|██████████| 370/370 [00:00<00:00, 7588.23it/s]


Extracting Dataset Version Zip to /content/florence2_od/ in florence2-od:: 100%|██████████| 372/372 [00:00<00:00, 7485.27it/s]


In [3]:
# for each of the dataset model formats (coco / paligemma / florence2_od)
# do a "ls -l" of the json files
!ls -l coco/train/*.json
!ls -l coco/test/*.json
!ls -l coco/valid/*.json

!ls -l florence2_od/test/*.json*
!ls -l florence2_od/train/*.json*
!ls -l florence2_od/valid/*.json*

!ls -l paligemma/dataset/*.json*

-rw-r--r-- 1 root root 1122675 Jul 23 00:33 coco/train/_annotations.coco.json
-rw-r--r-- 1 root root 153472 Jul 23 00:33 coco/test/_annotations.coco.json
-rw-r--r-- 1 root root 314313 Jul 23 00:33 coco/valid/_annotations.coco.json
-rw-r--r-- 1 root root 21993 Jul 23 00:33 florence2_od/test/annotations.jsonl
-rw-r--r-- 1 root root 160106 Jul 23 00:33 florence2_od/train/annotations.jsonl
-rw-r--r-- 1 root root 45081 Jul 23 00:33 florence2_od/valid/annotations.jsonl
-rw-r--r-- 1 root root  24860 Jul 23 00:33 paligemma/dataset/_annotations.test.jsonl
-rw-r--r-- 1 root root 180953 Jul 23 00:33 paligemma/dataset/_annotations.train.jsonl
-rw-r--r-- 1 root root  50942 Jul 23 00:33 paligemma/dataset/_annotations.valid.jsonl


In [4]:
import os

def count_jpg_files(directory):
    return len([f for f in os.listdir(directory) if f.endswith('.jpg')])

directories = [
    "/content/coco/train",
    "/content/coco/test",
    "/content/coco/valid",
    "/content/florence2_od/test",
    "/content/florence2_od/train",
    "/content/florence2_od/valid",
    "/content/paligemma/dataset"
]

for directory in directories:
    try:
        count = count_jpg_files(directory)
        print(f"{directory}: {count} files")
    except FileNotFoundError:
        print(f"{directory}: Directory not found")


/content/coco/train: 255 files
/content/coco/test: 36 files
/content/coco/valid: 73 files
/content/florence2_od/test: 36 files
/content/florence2_od/train: 255 files
/content/florence2_od/valid: 73 files
/content/paligemma/dataset: 364 files


In [5]:
import os
import json
from datasets import Dataset, Features, Image, Sequence, Value, ClassLabel, DatasetDict

def load_coco_annotations(annotation_file, image_dir):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # Create id to filename mapping
    id_to_filename = {img['id']: img['file_name'] for img in data['images']}

    # Create category id to name mapping
    id_to_category = {cat['id']: cat['name'] for cat in data['categories']}

    # Process annotations
    image_annotations = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        if image_id not in image_annotations:
            image_annotations[image_id] = []
        image_annotations[image_id].append(ann)

    # Create dataset entries
    dataset_entries = []
    for img in data['images']:
        entry = {
            'image_id': img['id'],
            'image': os.path.join(image_dir, img['file_name']),
            'width': img['width'],
            'height': img['height'],
            'objects': {
                'id': [],
                'area': [],
                'bbox': [],
                'category': []
            }
        }

        if img['id'] in image_annotations:
            for ann in image_annotations[img['id']]:
                entry['objects']['id'].append(ann['id'])
                entry['objects']['area'].append(ann['area'])
                entry['objects']['bbox'].append(ann['bbox'])
                entry['objects']['category'].append(id_to_category[ann['category_id']])

        dataset_entries.append(entry)

    return dataset_entries

# Load annotations
train_data = load_coco_annotations('/content/coco/train/_annotations.coco.json', '/content/coco/train')
test_data = load_coco_annotations('/content/coco/test/_annotations.coco.json', '/content/coco/test')
valid_data = load_coco_annotations('/content/coco/valid/_annotations.coco.json', '/content/coco/valid')

# Define features
features = Features({
    'image_id': Value('int64'),
    'image': Image(),
    'width': Value('int32'),
    'height': Value('int32'),
    'objects': {
        'id': Sequence(Value('int64')),
        'area': Sequence(Value('int64')),
        'bbox': Sequence(Sequence(Value('float32'), length=4)),
        'category': Sequence(ClassLabel(names=['RBC', 'WBC', 'Platelets']))
    }
})

# Create datasets
coco_train_dataset = Dataset.from_list(train_data, features=features)
coco_test_dataset = Dataset.from_list(test_data, features=features)
coco_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
coco_ds = DatasetDict({
    'train': coco_train_dataset,
    'test': coco_test_dataset,
    'validation': coco_valid_dataset
})

In [6]:
# Push to Hub
coco_ds.push_to_hub("dwb2023/bccd-coco")

# Print some information about the dataset
print(f"Dataset splits: {coco_ds.keys()}")
print(f"Number of training examples: {len(coco_ds['train'])}")
print(f"Number of testing examples: {len(coco_ds['test'])}")
print(f"Number of validation examples: {len(coco_ds['validation'])}")
print(f"Features: {coco_ds['train'].features}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='int64', id=None), 'image': Image(mode=None, decode=True, id=None), 'width': Value(dtype='int32', id=None), 'height': Value(dtype='int32', id=None), 'objects': {'id': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'area': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'bbox': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=4, id=None), length=-1, id=None), 'category': Sequence(feature=ClassLabel(names=['RBC', 'WBC', 'Platelets'], id=None), length=-1, id=None)}}


In [7]:
import os
import json
from datasets import Dataset, Features, Image, Value, Sequence, DatasetDict

def load_coco_annotations(annotation_file, image_dir):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # Create image id to annotations mapping
    image_annotations = {img['id']: {'image': img, 'annotations': []} for img in data['images']}
    for ann in data['annotations']:
        image_annotations[ann['image_id']]['annotations'].append(ann)

    # Create dataset entries
    dataset_entries = []
    for img_id, img_data in image_annotations.items():
        entry = {
            'image_id': img_data['image']['file_name'],
            'image': os.path.join(image_dir, img_data['image']['file_name']),
            'width': img_data['image']['width'],
            'height': img_data['image']['height'],
            'annotations': {
                'image': img_data['image'],
                'categories': {
                    'id': [cat['id'] for cat in data['categories']],
                    'name': [cat['name'] for cat in data['categories']],
                    'supercategory': [cat['supercategory'] for cat in data['categories']]
                },
                'annotations': {
                    'id': [ann['id'] for ann in img_data['annotations']],
                    'image_id': [ann['image_id'] for ann in img_data['annotations']],
                    'category_id': [ann['category_id'] for ann in img_data['annotations']],
                    'bbox': [ann['bbox'] for ann in img_data['annotations']],
                    'area': [ann['area'] for ann in img_data['annotations']],
                    'segmentation': [ann['segmentation'] for ann in img_data['annotations']],
                    'iscrowd': [ann['iscrowd'] for ann in img_data['annotations']]
                }
            }
        }
        dataset_entries.append(entry)

    return dataset_entries

# Load annotations
train_data = load_coco_annotations('/content/coco/train/_annotations.coco.json', '/content/coco/train')
test_data = load_coco_annotations('/content/coco/test/_annotations.coco.json', '/content/coco/test')
valid_data = load_coco_annotations('/content/coco/valid/_annotations.coco.json', '/content/coco/valid')

# Define features
features = Features({
    'image_id': Value('string'),
    'image': Image(),
    'width': Value('int32'),
    'height': Value('int32'),
    'annotations': {
        'image': {
            'id': Value('int64'),
            'license': Value('int64'),
            'file_name': Value('string'),
            'height': Value('int64'),
            'width': Value('int64'),
            'date_captured': Value('string')
        },
        'categories': {
            'id': Sequence(Value('int64')),
            'name': Sequence(Value('string')),
            'supercategory': Sequence(Value('string'))
        },
        'annotations': {
            'id': Sequence(Value('int64')),
            'image_id': Sequence(Value('int64')),
            'category_id': Sequence(Value('int64')),
            'bbox': Sequence(Sequence(Value('float32'), length=4)),
            'area': Sequence(Value('float32')),
            'segmentation': Sequence(Sequence(Value('float32'))),
            'iscrowd': Sequence(Value('int64'))
        }
    }
})

# Create datasets
coco_train_dataset = Dataset.from_list(train_data, features=features)
coco_test_dataset = Dataset.from_list(test_data, features=features)
coco_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
coco_dataset = DatasetDict({
    'train': coco_train_dataset,
    'test': coco_test_dataset,
    'validation': coco_valid_dataset
})

In [8]:
# Push to Hub
coco_dataset.push_to_hub("dwb2023/roboflow100-bccd-coco")

# Print some information about the dataset
print(f"Dataset splits: {coco_dataset.keys()}")
print(f"Number of training examples: {len(coco_dataset['train'])}")
print(f"Number of testing examples: {len(coco_dataset['test'])}")
print(f"Number of validation examples: {len(coco_dataset['validation'])}")
print(f"Features: {coco_dataset['train'].features}")

# Print a sample entry
print("\nSample entry:")
print(coco_dataset['train'][0])

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None), 'width': Value(dtype='int32', id=None), 'height': Value(dtype='int32', id=None), 'annotations': {'image': {'id': Value(dtype='int64', id=None), 'license': Value(dtype='int64', id=None), 'file_name': Value(dtype='string', id=None), 'height': Value(dtype='int64', id=None), 'width': Value(dtype='int64', id=None), 'date_captured': Value(dtype='string', id=None)}, 'categories': {'id': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'name': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'supercategory': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, 'annotations': {'id': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'image_id': Sequence(feature=V

In [9]:
import os
import json
from datasets import Dataset, Features, Image, Value, DatasetDict

def load_florence2_annotations(annotation_file, image_dir):
    dataset_entries = []
    with open(annotation_file, 'r') as f:
        for line in f:
            ann = json.loads(line)
            entry = {
                'image_id': ann['image'],
                'image': os.path.join(image_dir, ann['image']),
                'annotations': ann  # Keep all original fields
            }
            dataset_entries.append(entry)
    return dataset_entries

# Load annotations
train_data = load_florence2_annotations('/content/florence2_od/train/annotations.jsonl', '/content/florence2_od/train')
test_data = load_florence2_annotations('/content/florence2_od/test/annotations.jsonl', '/content/florence2_od/test')
valid_data = load_florence2_annotations('/content/florence2_od/valid/annotations.jsonl', '/content/florence2_od/valid')

# Define features
features = Features({
    'image_id': Value('string'),
    'image': Image(),
    'annotations': {
        'image': Value('string'),
        'prefix': Value('string'),
        'suffix': Value('string')
    }
})

# Create datasets
florence2_train_dataset = Dataset.from_list(train_data, features=features)
florence2_test_dataset = Dataset.from_list(test_data, features=features)
florence2_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
florence2_dataset = DatasetDict({
    'train': florence2_train_dataset,
    'test': florence2_test_dataset,
    'validation': florence2_valid_dataset
})

In [10]:
# Push to Hub
florence2_dataset.push_to_hub("dwb2023/roboflow100-bccd-florence2")

# Print some information about the dataset
print(f"Dataset splits: {florence2_dataset.keys()}")
print(f"Number of training examples: {len(florence2_dataset['train'])}")
print(f"Number of testing examples: {len(florence2_dataset['test'])}")
print(f"Number of validation examples: {len(florence2_dataset['validation'])}")
print(f"Features: {florence2_dataset['train'].features}")

# Print a sample entry
print("\nSample entry:")
print(florence2_dataset['train'][0])

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None), 'annotations': {'image': Value(dtype='string', id=None), 'prefix': Value(dtype='string', id=None), 'suffix': Value(dtype='string', id=None)}}

Sample entry:
{'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7A5A7B1417B0>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': '<OD>', 'suffix': 'RBC<loc_756><loc_406><loc_958><loc_631>RBC<loc_820><loc_623><loc_995><loc_825>RBC<loc_150><loc_327><loc_355><loc_508>RBC<loc_283><loc_685><loc_283><loc_685>RBC<loc_400><loc_417><loc_588><loc_640>RBC<loc_817><loc_2><loc_999><loc_248>RBC<loc_50><loc_10><loc_208><loc_204>RBC<loc_206><loc_46><loc_3

In [12]:
import os
import json
from datasets import Dataset, Features, Image, Value, DatasetDict

def load_paligemma_annotations(annotation_file, image_dir):
    dataset_entries = []
    with open(annotation_file, 'r') as f:
        for line in f:
            ann = json.loads(line)
            entry = {
                'image_id': ann['image'],
                'image': os.path.join(image_dir, ann['image']),
                'annotations': ann  # Keep all original fields
            }
            dataset_entries.append(entry)
    return dataset_entries

# Load annotations
train_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.train.jsonl', '/content/paligemma/dataset')
test_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.test.jsonl', '/content/paligemma/dataset')
valid_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.valid.jsonl', '/content/paligemma/dataset')

# Define features
features = Features({
    'image_id': Value('string'),
    'image': Image(),
    'annotations': {
        'image': Value('string'),
        'prefix': Value('string'),
        'suffix': Value('string')
    }
})

# Create datasets
paligemma_train_dataset = Dataset.from_list(train_data, features=features)
paligemma_test_dataset = Dataset.from_list(test_data, features=features)
paligemma_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
paligemma_dataset = DatasetDict({
    'train': paligemma_train_dataset,
    'test': paligemma_test_dataset,
    'validation': paligemma_valid_dataset
})

# Push to Hub
paligemma_dataset.push_to_hub("dwb2023/roboflow-bccd-paligemma")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dwb2023/roboflow-bccd-paligemma/commit/413146de1ef6f636fa78cbc385014af02604fab2', commit_message='Upload dataset', commit_description='', oid='413146de1ef6f636fa78cbc385014af02604fab2', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
# Print some information about the dataset
print(f"Dataset splits: {paligemma_dataset.keys()}")
print(f"Number of training examples: {len(paligemma_dataset['train'])}")
print(f"Number of testing examples: {len(paligemma_dataset['test'])}")
print(f"Number of validation examples: {len(paligemma_dataset['validation'])}")
print(f"Features: {paligemma_dataset['train'].features}")

# Print a sample entry
print("\nSample entry:")
print(paligemma_dataset['train'][0])

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None), 'annotations': {'image': Value(dtype='string', id=None), 'prefix': Value(dtype='string', id=None), 'suffix': Value(dtype='string', id=None)}}

Sample entry:
{'image_id': 'BloodImage_00327_jpg.rf.dc708dec2caf7249ddca49d41055ca4c.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7A5AAF6B9BD0>, 'annotations': {'image': 'BloodImage_00327_jpg.rf.dc708dec2caf7249ddca49d41055ca4c.jpg', 'prefix': 'detect Platelets ; RBC ; WBC', 'suffix': '<loc0232><loc0169><loc0296><loc0233> Platelets ; <loc0217><loc0529><loc0302><loc0606> Platelets'}}


In [14]:
# Search for same image across datasets
image_id_to_find = 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg'

# Function to find a record by image_id
def find_record_by_image_id(dataset, image_id):
    for record in dataset['train']:
        if record['image_id'] == image_id:
            return record
    return None

In [15]:
# Find the COCO record
record = find_record_by_image_id(coco_ds, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record not found


In [16]:
# Find the RoboFlow COCO record
record = find_record_by_image_id(coco_dataset, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record found: {'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7A5AAF7F6470>, 'width': 640, 'height': 480, 'annotations': {'image': {'id': 224, 'license': 1, 'file_name': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'height': 480, 'width': 640, 'date_captured': '2022-08-30T10:17:33+00:00'}, 'categories': {'id': [0, 1, 2, 3], 'name': ['bccd', 'Platelets', 'RBC', 'WBC'], 'supercategory': ['none', 'bccd', 'bccd', 'bccd']}, 'annotations': {'id': [3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073], 'image_id': [224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224], 'category_id': [2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 3], 'bbox': [[484.0, 195.0, 129.0, 108.0], [525.0, 299.0, 112.0, 97.0], [96.0, 157.0, 131.0, 87.0], [181.0, 329.0, 0.0, 0.0], [256.0, 200.0, 120.0, 107.0], [523.0, 1.0, 117.0, 118.0], [32.0, 5.0, 101.0, 93.0], [132.0, 22.0, 69.0, 123

In [17]:
# Find the Roboflow Florence2 record
record = find_record_by_image_id(florence2_dataset, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record found: {'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7A5AAF78C130>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': '<OD>', 'suffix': 'RBC<loc_756><loc_406><loc_958><loc_631>RBC<loc_820><loc_623><loc_995><loc_825>RBC<loc_150><loc_327><loc_355><loc_508>RBC<loc_283><loc_685><loc_283><loc_685>RBC<loc_400><loc_417><loc_588><loc_640>RBC<loc_817><loc_2><loc_999><loc_248>RBC<loc_50><loc_10><loc_208><loc_204>RBC<loc_206><loc_46><loc_314><loc_302>RBC<loc_528><loc_677><loc_670><loc_925>Platelets<loc_2><loc_752><loc_75><loc_854>Platelets<loc_109><loc_429><loc_184><loc_531>WBC<loc_286><loc_2><loc_664><loc_319>'}}


In [18]:
# Find the Roboflow Paligemma record
record = find_record_by_image_id(paligemma_dataset, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record found: {'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7A5AAF7F4D90>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': 'detect Platelets ; RBC ; WBC', 'suffix': '<loc0416><loc0774><loc0646><loc0980> RBC ; <loc0637><loc0840><loc0844><loc1019> RBC ; <loc0334><loc0153><loc0520><loc0363> RBC ; <loc0701><loc0289><loc0701><loc0289> RBC ; <loc0426><loc0409><loc0654><loc0601> RBC ; <loc0002><loc0836><loc0253><loc1023> RBC ; <loc0010><loc0051><loc0209><loc0212> RBC ; <loc0046><loc0211><loc0309><loc0321> RBC ; <loc0693><loc0540><loc0947><loc0686> RBC ; <loc0770><loc0001><loc0874><loc0076> Platelets ; <loc0439><loc0112><loc0544><loc0188> Platelets ; <loc0002><loc0292><loc0326><loc0680> WBC'}}
